In [1]:
# initialization
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import gensim.downloader
import re
from sklearn.preprocessing import StandardScaler

import nltk
from nltk.corpus import stopwords  #stopwords
from nltk import word_tokenize,sent_tokenize # tokenizing
from nltk.stem import PorterStemmer,LancasterStemmer  # using the Porter Stemmer and Lancaster Stemmer and others
from nltk.stem.snowball import SnowballStemmer
from nltk.stem import WordNetLemmatizer  # lammatizer from WordNet

import torch
import torch.nn as nn
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence, pad_sequence

from sklearn.metrics import accuracy_score

nltk.download('punkt')
nltk.download('stopwords')

np.random.seed(42)

[nltk_data] Downloading package punkt to /Users/fionchai/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/fionchai/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Import data

In [2]:
%%script false --no-raise-error # comment this if you dont have the dev_set

# import dataset
train_set = pd.read_csv('train.csv')

test_set = pd.read_csv('test.csv')

# from train_set sample development set
dev_set = train_set.sample(n=500, replace=False)

# remove dev set from train set
train_set = train_set.drop(dev_set.index)

# check
print(train_set.shape, dev_set.shape, test_set.shape)

# save to dataframe
dev_set.to_csv("dev_set.csv", index=False)
train_set.to_csv("train_set_modified.csv", index=False)

In [3]:
train_set = pd.read_csv('train_set_modified.csv')
dev_set = pd.read_csv('dev_set.csv')
test_set = pd.read_csv('test.csv')

## Preprocess data (averaging over word representations)

TODO: Try max pooling

TODO: Take representation of last word in LSTM

TODO: Use attention and perform weighted average?



In [4]:
# word2vec

# download the word2vec-google-news-300
w2v = gensim.downloader.load('word2vec-google-news-300')

In [5]:
# select four classes: 0, 1, 2, 3
# 4 and 5 will be OTHERS (4)

# for train_set
train_set.loc[train_set['label-coarse'] > 4, 'label-coarse'] = 4

# for dev_set
dev_set.loc[dev_set['label-coarse'] > 4, 'label-coarse'] = 4

# for test_set
test_set.loc[test_set['label-coarse'] > 4, 'label-coarse'] = 4


In [6]:
# neural network transforming the input for each word to its final vector representation
def token(sentence):

    # keep only english words
    sentence = re.sub("[^a-zA-Z]"," ",sentence)

    # converting to lower case and splitting

    # stop word removal
    words = sentence.split()
    stop_words = set(stopwords.words('english'))
    filtered_words = [word.lower() for word in words if word.lower() not in stop_words]
    filtered_sentence = ' '.join(filtered_words)

    if (len(filtered_sentence) == 0):
        filtered_sentence = sentence

    token = word_tokenize(filtered_sentence)
    return token

In [7]:
train_set['cleaned_text'] = train_set['text'].apply(token)

train_set.head()

,label-coarse,label-fine,text,cleaned_text
0,0,0,How did serfdom develop in and then leave Russ...,"[serfdom, develop, leave, russia]"
1,1,1,What films featured the character Popeye Doyle ?,"[films, featured, character, popeye, doyle]"
2,0,0,How can I find a list of celebrities ' real na...,"[find, list, celebrities, real, names]"
3,1,2,What fowl grabs the spotlight after the Chines...,"[fowl, grabs, spotlight, chinese, year, monkey]"
4,2,3,What is the full form of .com ?,"[full, form, com]"


In [8]:
max_length = train_set['cleaned_text'].str.len().max()

# get the embedding shape of the model
embed_shape = len(w2v['test'])
average_word_embeddings = []

for index, row in train_set.iterrows():

    sentence = row['cleaned_text']

    # get word embedding of each word
    word_embeddings = []

    for word in sentence:
        # check if the word is present in the model
        if word in w2v.key_to_index:
            word_embeddings.append(w2v[word])
        else:
             word_embeddings.append(np.zeros(shape=(embed_shape)))

    # perform averaging of word embeddings
    awe = np.mean(word_embeddings, axis = 0)
    average_word_embeddings.append(awe)

train_set['vector'] = average_word_embeddings

train_set.head()


,label-coarse,label-fine,text,cleaned_text,vector
0,0,0,How did serfdom develop in and then leave Russ...,"[serfdom, develop, leave, russia]","[-0.013671875, -0.05543518, 0.05633545, 0.2869..."
1,1,1,What films featured the character Popeye Doyle ?,"[films, featured, character, popeye, doyle]","[-0.00927734375, 0.07685546875, -0.05764770507..."
2,0,0,How can I find a list of celebrities ' real na...,"[find, list, celebrities, real, names]","[0.029272461, 0.13002929, -0.021777343, 0.1398..."
3,1,2,What fowl grabs the spotlight after the Chines...,"[fowl, grabs, spotlight, chinese, year, monkey]","[0.06305949, 0.03805542, -0.08516184, 0.015625..."
4,2,3,What is the full form of .com ?,"[full, form, com]","[-0.036621094, -0.007965088, -0.08154297, 0.04..."


In [9]:
def aggregate_representations(dataset):
  dataset['cleaned_text'] = dataset['text'].apply(token)
  max_length = dataset['cleaned_text'].str.len().max()

  # get the embedding shape of the model
  embed_shape = len(w2v['test'])
  average_word_embeddings = []

  for index, row in dataset.iterrows():

      sentence = row['cleaned_text']

      # get word embedding of each word
      word_embeddings = []

      for word in sentence:
          # check if the word is present in the model
          if word in w2v.key_to_index:
              word_embeddings.append(w2v[word])
          else:
              word_embeddings.append(np.zeros(shape=(embed_shape)))

      # perform averaging of word embeddings
      awe = np.mean(word_embeddings, axis = 0)

      average_word_embeddings.append(awe)


  dataset['vector'] = average_word_embeddings
  display(dataset[dataset['vector'].isna()])

  dataset = pd.concat([train_set, train_set['vector'].apply(lambda x: pd.Series(x))], axis=1)

  return dataset

dev_set = aggregate_representations(dev_set)
train_set = aggregate_representations(train_set)

# train_set

,label-coarse,label-fine,text,cleaned_text,vector


,label-coarse,label-fine,text,cleaned_text,vector


### Prepare data for training

In [10]:
def encode_y_label(df, num_classes): # one hot encoding
  encoded_labels = df['label-coarse'].apply(lambda label: list(np.eye(num_classes)[label]))
  df['encoded-coarse-label'] = encoded_labels
  return df
  # print(df['encoded_coarse_label'])

train_set = encode_y_label(train_set, 5)
dev_set = encode_y_label(dev_set, 5)

In [11]:
# split into X and y
X_train = train_set.drop(columns=['label-coarse', 'label-fine', 'text', 'cleaned_text', 'vector', 'encoded-coarse-label']).to_numpy()
y_train = train_set['encoded-coarse-label']

X_dev = dev_set.drop(columns=['label-coarse', 'label-fine', 'text', 'cleaned_text', 'vector', 'encoded-coarse-label']).to_numpy()
y_dev = dev_set['encoded-coarse-label']

# without one hot
# X_train = train_set.drop(columns=['label-coarse', 'label-fine', 'text', 'cleaned_text', 'vector']).to_numpy()
# y_train = train_set['label-coarse']

# X_dev = dev_set.drop(columns=['label-coarse', 'label-fine', 'text', 'cleaned_text', 'vector']).to_numpy()
# y_dev = dev_set['label-coarse']

# SCALING?

# mini-batch strategy
batch_size = 32

# def collate_fn(data):
#     x, y = data
#     zipped = zip(x, y)
#     return list(zipped)

class CustomDataset(Dataset):
    def __init__(self, X, y):
        self.X =torch.tensor(X, dtype=torch.float64)
        self.y =torch.tensor(y)

    def __len__(self):
        return len(self.y)

    def __getitem__(self,idx):
        return self.X[idx], self.y[idx]

train_data = CustomDataset(X_train, y_train)
dev_data = CustomDataset(X_dev, y_dev)

train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
dev_loader = DataLoader(dev_data, batch_size=batch_size, shuffle=True)

##  LSTM model

In [18]:

HIDDEN_SIZE = 128
OUTPUT_SIZE = 5
NUM_LAYERS = 1


class simple_lstm(nn.Module):
    def __init__(
                    self, 
                    input_size= 300, 
                    hidden_size= HIDDEN_SIZE, 
                    output_size= OUTPUT_SIZE,
                    num_layers= NUM_LAYERS,
                   # dropout_rate= 0.2, 
                    bidirectional= True, 
                ):
        
        super().__init__()
        # Model body
        self.lstm = nn.LSTM(input_size= input_size, hidden_size= hidden_size, bidirectional= bidirectional, num_layers= num_layers, batch_first= True)
        
        # Model head
        self.head = nn.Sequential(
            nn.Linear((2 if bidirectional else 1) * hidden_size, output_size),
           # nn.Dropout(dropout_rate),
            nn.Softmax(dim= 2)
        )
    
    def __call__(self, input):

        return self.head(self.lstm(input)[0])

In [19]:
# fed into the softmax classifier to predict the final label
class LSTMModel(nn.Module):
    def __init__(self, input_d, hidden_d, layer_d, output_d):
        super(LSTMModel, self).__init__()

        self.hidden_dim = hidden_d
        self.layer_dim = layer_d

        # LSTM model
        # self.lstm = nn.LSTM(input_d, hidden_d, layer_d, batch_first=True)

        self.lstm = nn.LSTM(input_d, hidden_d, layer_d)

        self.head = nn.Sequential(
            nn.Linear(hidden_d, output_d),
            nn.Softmax(dim = 1) # softmax activation for label prediction
        )
        
    def forward(self, x, require_grad = True):
        '''
        lstm(input,(h_0, c_0)): shape for each param
        input = (batch_size, seq_len, features) (with batch_first = True)
        h_0 = (D * num_layers, batch_size, project_size or hidden size)
        c_0 = (D * num_layers, batch_size, project_size or hidden size)
        D = 2 if is bidirectional
        '''

        # 3D TENSOR
        # x = x.reshape(x.shape[0], 1, 300).float() # convert to 3D tensor

        # if require_grad:
        #   h0 = torch.zeros(self.layer_dim, x.size(0), self.hidden_dim).requires_grad_()
        #   c0 = torch.zeros(self.layer_dim, x.size(0), self.hidden_dim).requires_grad_()
        # else:
        #   h0 = torch.zeros(self.layer_dim, x.size(0), self.hidden_dim, requires_grad=False)
        #   c0 = torch.zeros(self.layer_dim, x.size(0), self.hidden_dim, requires_grad=False)

        x = x.float()

        # 2D TENSOR
        if require_grad:
          h0 = torch.zeros(self.layer_dim, self.hidden_dim).requires_grad_()
          c0 = torch.zeros(self.layer_dim, self.hidden_dim).requires_grad_()
        else:
          h0 = torch.zeros(self.layer_dim, self.hidden_dim, requires_grad=False)
          c0 = torch.zeros(self.layer_dim, self.hidden_dim, requires_grad=False)


        out, (hn, cn) = self.lstm(x, (h0.detach(), c0.detach()))
        
        out = self.head(out)
        
        return out

input_dim = embed_shape
hidden_dim = 128
output_dim = 5
layer_dim = 300

model = simple_lstm(input_dim, 128, 5, 1)
criterion = nn.CrossEntropyLoss()
learning_rate = 0.001
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

In [29]:
# REFERENCE
num_epochs = 100
best_accuracy = 0.0
max_patience = 5
current_patience = 0

for epoch in range(num_epochs):
  model.train()
  total_training_loss = 0.0
  total_dev_loss = 0.0

  for x_batch, y_batch in train_loader:
    x_batch = x_batch.unsqueeze(1).float() # convert to 3D tensor
    outputs = model(x_batch) # forward pass
    optimizer.zero_grad() # calculate the gradient, manually setting to 0

    # obtain the loss function
    # print(outputs.squeeze(1).shape)
    # print(y_batch.shape)
    loss = criterion(outputs.squeeze(1), y_batch)
    if torch.isnan(loss):
       print("1")

    loss.backward() #calculates the loss of the loss function

    optimizer.step() #improve from loss, i.e backprop

    total_training_loss += loss.item()

  average_loss = total_training_loss / len(train_loader)
  print("Epoch: %d, training set loss: %1.5f" % (epoch, average_loss))

  # evaluate on dev set
  model.eval()
  with torch.no_grad():
    all_true_labels = []
    all_predicted_labels = []
    for inputs in dev_loader:
      x_batch, y_batch = inputs
      x_batch = x_batch.unsqueeze(1).float()

      # forward pass
      y_pred = model(x_batch)
      loss = criterion(y_pred.squeeze(1), y_batch)

      total_dev_loss += loss.item()

      print(y_pred.shape)

      predicted_labels = torch.argmax(y_pred.squeeze(1), dim=1)

      print("1", predicted_labels)

      y_batch = torch.argmax(y_batch, dim=1)

      print(y_batch)

      all_true_labels.extend(y_batch.tolist())
      all_predicted_labels.extend(predicted_labels.tolist())

    average_dev_loss = total_dev_loss / len(dev_loader)
    print("Epoch: %d, dev set loss: %1.5f" % (epoch, average_dev_loss))

  def calculate_accuracy(y_true, y_pred):
      y_true = y_true.detach().cpu().numpy()
      y_pred = y_pred.detach().cpu().numpy()
      return accuracy_score(y_true, y_pred)

  accuracy = calculate_accuracy(torch.tensor(all_true_labels), torch.tensor(all_predicted_labels))
  print(f"Accuracy: {accuracy * 100:.2f}%")

  if accuracy > best_accuracy:
    best_accuracy = accuracy
    best_model = model.state_dict()
  else:
    current_patience += 1
    if current_patience >= max_patience:
        print(f"Early stopping after {epoch} epochs.")
        break


Epoch: 0, training set loss: 1.13945
torch.Size([32, 1, 5])
1 tensor([4, 1, 4, 0, 4, 4, 1, 1, 3, 2, 0, 1, 1, 4, 2, 1, 1, 4, 3, 4, 3, 0, 1, 4,
        1, 1, 4, 3, 0, 1, 4, 1])
tensor([4, 1, 4, 0, 4, 4, 3, 1, 0, 2, 0, 1, 1, 4, 2, 1, 1, 3, 3, 4, 3, 2, 1, 4,
        1, 1, 4, 3, 0, 1, 4, 1])
torch.Size([32, 1, 5])
1 tensor([4, 4, 4, 0, 4, 4, 4, 0, 3, 3, 4, 3, 3, 3, 3, 3, 4, 4, 3, 3, 4, 1, 1, 4,
        1, 1, 4, 4, 2, 3, 4, 1])
tensor([4, 0, 4, 0, 4, 0, 4, 1, 3, 4, 4, 3, 1, 0, 3, 3, 4, 4, 3, 3, 4, 1, 4, 3,
        1, 1, 1, 0, 2, 3, 4, 1])
torch.Size([32, 1, 5])
1 tensor([1, 4, 3, 4, 3, 1, 3, 3, 3, 3, 4, 1, 3, 3, 0, 3, 0, 4, 3, 4, 1, 3, 1, 0,
        3, 1, 4, 1, 1, 3, 2, 0])
tensor([1, 4, 3, 1, 0, 1, 3, 3, 0, 3, 4, 1, 1, 3, 0, 3, 0, 4, 4, 3, 1, 3, 1, 3,
        1, 1, 4, 1, 1, 3, 2, 0])
torch.Size([32, 1, 5])
1 tensor([1, 3, 4, 4, 3, 3, 4, 4, 3, 1, 1, 3, 0, 4, 4, 1, 4, 3, 4, 0, 3, 1, 4, 0,
        4, 0, 3, 4, 4, 0, 0, 3])
tensor([1, 3, 4, 4, 0, 3, 4, 4, 3, 1, 0, 3, 0, 4, 4, 1, 4, 4, 4, 0, 3, 1

In [ ]:
print(train_data[0][0].unsqueeze(0).float().shape)

torch.Size([1, 2100])


In [ ]:
model = simple_lstm(input_dim, hidden_dim, 5, 1)

print(train_data[0][0])
print(model(train_data[0][0].unsqueeze(0).unsqueeze(1).float()))
print(train_data[1][0])
print(model(train_data[1][0].unsqueeze(0).unsqueeze(1).float()))

tensor([-1.3672e-02, -5.5435e-02,  5.6335e-02,  2.8699e-01, -1.2817e-02,
         1.0815e-01,  7.6268e-02, -1.8945e-01,  2.6489e-02,  9.4727e-02,
        -8.2932e-02, -1.2927e-01, -7.5562e-02,  1.3550e-01, -2.1545e-01,
         6.3965e-02,  2.3438e-02,  1.0641e-01,  8.1299e-02, -5.7861e-02,
         1.5720e-01,  6.5674e-02,  9.9609e-02, -1.8921e-02,  8.1805e-02,
         1.9151e-01, -5.1636e-02,  3.1158e-02, -4.8569e-02,  9.5520e-03,
        -5.0186e-02, -9.7717e-02, -1.5649e-01, -7.2021e-02, -1.9183e-01,
         4.8706e-02, -3.7720e-02, -4.7913e-02,  1.6888e-01,  1.0187e-01,
         6.2317e-02,  4.6509e-02,  3.3691e-01,  2.9449e-03,  1.5746e-01,
        -2.0752e-03,  3.5461e-02, -1.3672e-02, -1.7914e-01,  1.5430e-01,
        -6.2622e-02,  1.2259e-01,  4.7836e-03, -2.2766e-02, -7.5073e-03,
        -3.7155e-02, -2.4939e-01, -1.7018e-01,  1.0658e-02, -1.3232e-01,
         1.6052e-02,  7.6965e-02, -3.2574e-02, -5.1392e-02, -1.5468e-01,
        -8.0566e-02,  5.2368e-02, -1.0925e-02, -8.2

In [ ]:
# print(model(train_data[0:2][0].unsqueeze(0).float()).squeeze(0))
print(criterion(model(train_data[0:2][0].unsqueeze(0).float()).squeeze(0), train_data[0:2][1]))

print(model(train_data[0:2][0].unsqueeze(0).float()).squeeze(0).shape)
print( train_data[0:2][1].shape)

test_pred = torch.tensor(np.array([[0.5, 0.4, 0.1, 0, 0, 0, 0, 0, 0] for _ in range(150)])).float()
print(test_pred.shape)
print(torch.tensor(np.array([[1, 0, 0, 0, 0, 0, 0, 0, 0] for _ in range(150)])).shape)
nn.functional.cross_entropy(test_pred, torch.tensor(np.array([0 for _ in range(150)])).long()), nn.functional.cross_entropy(test_pred, torch.tensor(np.array([[1, 0, 0, 0, 0, 0, 0, 0, 0] for _ in range(150)])).float())

check
tensor(1.6062, dtype=torch.float64, grad_fn=<DivBackward1>)
check
torch.Size([2, 5])
torch.Size([2, 5])
torch.Size([150, 9])
torch.Size([150, 9])


(tensor(1.8269), tensor(1.8269))

In [ ]:
model(train_data[1][0].unsqueeze(0).unsqueeze(0).float()).shape


check


torch.Size([1, 1, 5])

- aggregation: max pooling, last word
- number of layers
- hidden dimensions
- learning rate
- mini-batch size